# the goal of this notebook is to find the DaqeventNumber that satisfy the codition of N_pulse(>500mV)&Three_hit(>500mV)_in_a_line.

Based my analysis on the past several weeks for beam muon event, Mike suggest to use the cut of three in a line and number of pulse in a event to filter out the shower event. Right now, the timing calibration related to signal travel through cable hasn't done yet. So, I can't use the timing cut that is less than 4 ns between layers for muon while traveling in a three in a line path as an indicaiton of beam muon.  My plan is to make eventdisplay for all of those events(The program hasn't finished yet) to seek out properties of beam muon in a event. For example, I will check the geometry distribution of hit within the event at above. This notebook is used for the powerpoint 3-10 beammuon.
This notebook is modified based on datacut.py
3/15/22 -Collin



In [1]:
import ROOT as r


Welcome to JupyROOT 6.24/07


In [7]:
import numpy as np

In [2]:
from DetectorGeometry import *

In [3]:
# Create Data Class
class DataHandler():
    def __init__(self, dataPath, debug=False):
        self.data = self.initializeData(dataPath)
        self.debug = debug
    # Create TChain with all data you will need for your analysis
    # wildcards can be used in datapath
    def initializeData(self, datapath):
       treeChain = r.TChain('t')
       treeChain.Add(datapath)
       return treeChain


    # Print out certain datafile statistics
    # FIXME: Implement way to viewData or get rid of function
    def viewData(self):
        return

    # For a millicharged particle it is expected that the ratio between the maximum and minimum nPE is less than 10
    # This function removes events that don't satisfy this cut
    def npeCut(self, event, cut_ratio):
        # FIXME: How do we reconcile the issue where some values are negative
        nPEratio = max(event.nPE)/min(event.nPE)
        if nPEratio > cut_ratio:
            return
        if self.debug:
            print(event.nPE)
            print(nPEratio)
        return event
    
    # Removes events that have a hit in the cosmic veto panels labeled by channel 68, 69, 70, 72, 73, 74.
    def cosmicPanelVeto(self, event):
        # Getting list of channels for each event
        channels = event.chan
        # vetoPanels is defined in DetectorGeometry.py
        if any(veto in channels for veto in vetoPanels):
            return
        if self.debug:
            print(channels)
        return event


    # Removes events that have more than a single hit per layer, ensuring particle travels in straight line
    def singleHitPerLayer(self, event):
        layers = event.layer
        # Sets have no repeated values so if the lenghts are not the same, there is some repeated value
        if len(set(layers)) != len(layers):
            return
        if self.debug:
            print(layers)
        return event


    #MuonSelection is base on NeHa's idea
    #this will not work in run 591 because there is not much hit in the end pannels
    def muonSelection(self,event, height_threshold):
        
        height_list = event.height
        chan_list = event.chan

        # if there is no hit on the end pannels, then we can't claims that this 
        # event is muon event
        if 71 not in chan_list:
            return
        elif 75 not in chan_list:
            return
        frontPannelHeight = []
        backPannelHeight = []
        #She claim if both end pannals has a big hit(>1.1V), then it is muon event
        for chan, height in zip(chan_list,height_list):

            if chan == 71:  
                frontPannelHeight.append(height)
            elif chan == 75:
                backPannelHeight.append(height)
            
            if any(x >= height_threshold for x in frontPannelHeight) and any(x >= height_threshold for x in backPannelHeight):
                return event
            else:
                return

    def npeCheck(self,event):
        nPE_list = event.nPE
        DAQNum = event.DAQEventNumber
        #nPEthreashold is define in DetectorGeometry.py
        if min(nPE_list) >= nPEthreashold:
            #print(DAQNum)
            return event
        else:
            return None
        
    #use to restrict the number of pulse
    def NumPulse(self,event):
        height_list = event.height
        count = 0
        for pulse in height_list:
            #pulseHeight is define in 
            if pulse >= pulseHeight:
                count += 1
        if count == NumOfbigpulse:
            return event
        else:
            return None
            
        

    #this method is used with ThreeInLine()
    def layerCheck(self,lists):
        i = 0 
        for list in lists:
            if len(set(list)) >= 3: #return true if it contain 3 unique layers(3 in a line)
                i += 1
        return i


    
    def ThreeInLine(self,event):
        #print(event)
        row_list = event.row
        column_list = event.column
        layer_list = event.layer
        height_list = event.height

        chan_list = event.chan

        # four in a line is very rare impossible to find in run 591
        # use three in a line instead!
        #if len(layer_list) != 6: return  #num of pulse in a evnet
        #if len(layer_list) != 6: return  #num of pulse in a evnet
        #else:pass
 
        if len(set(layer_list)) >= 3:  #three in a line more than 10 pulse reject
            pass
        else:
            return None
        
        #for run 588 and 591, ch0 is in saturation
        
        for chan,height in zip(chan_list,height_list):
            if chan == 0 and height > 1000: return None
            else: continue
                
        # if the number of pulse increase layer by layer than it is shower

        
        # the list at below save the information about the pulse passing through difference layers
        # For example, List0 is used for saving the infomation pulse passing through the position of
        # row is three and column is one. "0" correspond to the position of the 0th channal, whose location
        # is at the third row and first column.

        #numbers at below are index number. So 3rd(index) row mean the 4th row in my notation
        list0 = [] # correspond to 3rd row, 0st column
        list1 = [] # correspond to 3rd row, 1st column
        list2 = [] # correspond to 2rd row, 0th column
        list3 = [] # correspond to 2rd row, 1st column
        list4 = [] #list 4-7 should not contain any hit in run 591 due to the last Super model is yet to install
        list5 = []
        list6 = [] 
        list7 = [] 
        list8 = []
        list9 = []
        list10 = []
        list11 = []
        list12 = []
        list13 = []
        list14 = []
        list15 = []
      
        for row, column, layer, height in zip(row_list,column_list,layer_list,height_list):
            if row == 3 and column == 0 and height > 500:
                list0.append(layer)
            elif row == 3 and column == 1 and height > 500:
                list1.append(layer)
            elif row == 2 and column == 0 and height > 500:
                list2.append(layer)
            elif row == 2 and column == 1 and height > 500:
                list3.append(layer)
            elif row == 3 and column == 2 and height > 500:
                list4.append(layer)
            elif row == 3 and column == 3 and height > 500:
                list5.append(layer)
            elif row == 2 and column == 2 and height > 500:
                list6.append(layer)
            elif row == 2 and column == 3 and height > 500:
                list7.append(layer)
            elif row == 1 and column == 0 and height > 500:
                list8.append(layer)
            elif row == 1 and column == 1 and height > 500:
                list9.append(layer)
            elif row == 0 and column == 0 and height > 500:
                list10.append(layer)
            elif row == 0 and column == 1 and height > 500:
                list11.append(layer)
            elif row == 1 and column == 2 and height > 500:
                list12.append(layer)
            elif row == 1 and column == 3 and height > 500:
                list13.append(layer)
            elif row == 0 and column == 2 and height > 500:
                list14.append(layer)
            elif row == 0 and column == 3 and height > 500:
                list15.append(layer)
                
        # check where does the 4 in a row locate
        lists=[list0,list1,list2,list3,list4,list5,list6,list7,list8,list9,list10,list11,list12,list13,list14,list15]
        #print("lists:"+str(lists)) #for debugs
        i = self.layerCheck(lists) #i is the number list in lists are 3 in a row
        if i >= 1:
            #print(np.array(DAQNum))
            #return event, DAQNum
            return event
        else:
            return None
        
   
    
    def applyCuts(self, cuts):
        lc=len(cuts)
        cutData = [] 

        # Run over all events
        for event in self.data:
            # Run over all cuts in list given to applyCuts
            None_check = []
            for cut in cuts:

                event= cut(event) # Apply given cut
                if event is None: 
                    break
                None_check.append(event.DAQEventNumber)


            if None in None_check:
                break
            if len(None_check) == lc: # if there are two cuts then len(None_check) should be 2
                cutData.append(None_check[0])
                
            
        return cutData


In [4]:
if __name__ == "__main__":
    data = DataHandler('/store/user/mcarrigan/skim/muon/v29/MilliQan_Run588.*_MuonSkim_v29.root')

    cutData = data.applyCuts([data.NumPulse,data.ThreeInLine])  #use mutiple cuts
    print(len(cutData))
    print(cutData) #use DaqEventNumber to identify the event

7
[122534, 165576, 175580, 1984, 41285, 54792, 71223]


# The following code is used to either verify the code at above or extract the useful infomation to make event display.

In [8]:
#print out all pulse in the event

treeChain = r.TChain("t")
treeChain.Add('/store/user/mcarrigan/skim/muon/v29/MilliQan_Run588.*_MuonSkim_v29.root')
for event in treeChain:
    DM=event.DAQEventNumber
    if DM in cutData:
        print('eventNum:' + str(event.event))
        print('FileNum:' + str(event.fileNumber))
        print('time:'+str(np.array(event.timeFit)))
        print('layer:'+str(np.array(event.layer)))
        print('height:'+str(np.array(event.height)))
        print('chan:'+str(np.array(event.chan)))

eventNum:533
FileNum:123
time:[1286.597   1278.8792  1837.901   1909.3005  2453.229   2095.9385
 1325.2943  1313.1682   828.7241  1336.8239  2332.4236  2392.8337
 1328.0889   198.17296 1337.4321  1351.8223 ]
layer:[0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3]
height:[ 132.52475  1249.7932     36.69817    12.563418  249.78377    16.359346
 1250.955    1249.7755     33.603394 1249.852      21.953838  124.29739
 1250.619      48.65202  1252.6416   1250.0469  ]
chan:[ 2  3  3  3  3 17 18 19 32 34 34 34 35 48 51 57]
eventNum:575
FileNum:166
time:[1310.2587 1876.7395 1287.7375 1310.194  1653.2434 1995.7482  666.9468
  994.53   2189.5671 1325.0028 1321.1038 2226.886  1296.5458 1320.8354
 1749.7463 1309.6036 1325.5273 2444.2417 1314.4081]
layer:[0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 3 3 3]
height:[1247.7961      14.11011   1249.9452     290.13324     43.199097
   38.420765    30.452166    31.027428    65.67441     65.6705
  666.28046     21.062849  1250.1782     407.45737     15.6862135
 1251.0002    1250.8522 

In [9]:
#only print out the pulse for height > 500mV
treeChain = r.TChain("t")
treeChain.Add('/store/user/mcarrigan/skim/muon/v29/MilliQan_Run588.*_MuonSkim_v29.root')
for event in treeChain:
    DM=event.DAQEventNumber
    if DM in cutData:
        
        time_list=event.timeFit
        layer_list = event.layer
        height_list = event.height
        chan_list = event.chan
        
        time_print_list = []
        layer_print_list = []
        height_print_list = []
        chan_print_list = []
        
        for time,layer,height,chan in zip(time_list,layer_list,height_list,chan_list):
            if height >= pulseHeight:
                time_print_list.append(time)
                layer_print_list.append(layer)
                height_print_list.append(height)
                chan_print_list.append(chan)
        
        print('eventNum:' + str(event.event))
        print('FileNum:' + str(event.fileNumber))
        print('time:'+str(time_print_list))
        print('layer:'+str(layer_print_list))
        print('height:'+str(height_print_list))
        print('chan:'+str(chan_print_list))
        
        

eventNum:533
FileNum:123
time:[1278.879150390625, 1325.2943115234375, 1313.168212890625, 1336.8238525390625, 1328.0888671875, 1337.43212890625, 1351.822265625]
layer:[0, 1, 1, 2, 2, 3, 3]
height:[1249.793212890625, 1250.9549560546875, 1249.7755126953125, 1249.85205078125, 1250.6190185546875, 1252.6416015625, 1250.046875]
chan:[3, 18, 19, 34, 35, 51, 57]
eventNum:575
FileNum:166
time:[1310.2586669921875, 1287.737548828125, 1321.103759765625, 1296.5457763671875, 1309.6036376953125, 1325.52734375, 1314.4080810546875]
layer:[0, 0, 1, 1, 2, 3, 3]
height:[1247.796142578125, 1249.9451904296875, 666.2804565429688, 1250.17822265625, 1251.000244140625, 1250.8521728515625, 1250.30126953125]
chan:[12, 14, 29, 30, 46, 60, 62]
eventNum:579
FileNum:176
time:[1310.2586669921875, 1287.737548828125, 1321.103759765625, 1296.5457763671875, 1309.6036376953125, 1325.52734375, 1314.4080810546875]
layer:[0, 0, 1, 1, 2, 3, 3]
height:[1247.796142578125, 1249.9451904296875, 666.2804565429688, 1250.17822265625, 1